<a href="https://colab.research.google.com/github/praveenpoluri/CS5590-0004/blob/master/icp6/L4_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.



In [17]:
!pip install syft
!pip install numpy

     |████████████████████████████████| 215kB 4.9MB/s 
     |████████████████████████████████| 450kB 46.7MB/s 
     |████████████████████████████████| 1.4MB 40.0MB/s 
     |████████████████████████████████| 204kB 48.1MB/s 
     |████████████████████████████████| 256kB 43.1MB/s 
     |████████████████████████████████| 389kB 43.6MB/s 
     |████████████████████████████████| 71kB 14.3MB/s 
     |████████████████████████████████| 276kB 42.8MB/s 
     |████████████████████████████████| 51kB 16.9MB/s 
     |████████████████████████████████| 122kB 42.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
     

In [0]:
import torch as th

In [19]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [21]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [22]:
import syft as sy

W0630 22:23:25.880173 140207670339456 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0630 22:23:25.898572 140207670339456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)  # this line of code create an instance of th with its backend APIs modified with Syft functions

In [24]:
th.tensor([1,2,3,4,5]) # notice that Torch functionalities still behave the same

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob") # creates a virtual worker (a simulaion to interface to Bob machines)

In [26]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob) # send this data to bob

In [29]:
bob._objects

{97250586923: tensor([1, 2, 3, 4, 5])}

In [30]:
x

(Wrapper)>[PointerTensor | me:66993015086 -> bob:97250586923]

In [31]:
x.location

<VirtualWorker id:bob #objects:1>

In [32]:

x.id_at_location

97250586923

In [33]:

x.id

66993015086

In [34]:
x.owner

<VirtualWorker id:me #objects:0>

In [35]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [36]:
x=x.get()
x

tensor([1, 2, 3, 4, 5])

In [37]:

bob._objects

{}

# Project: Experience with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
# create a second Virtual worker and call it Alice
alice = sy.VirtualWorker(hook, id="alice")


In [0]:
# 1 - create some data (a tensor)
# 2- send the data to bob and alice
x = th.tensor([1,2,3,4,5])


In [0]:
# notice what does the send fucntion on two workers return; a multi-pointer
x_ptr =  x.send(alice,bob)

In [42]:
alice._objects

{42439475104: tensor([1, 2, 3, 4, 5])}

In [44]:
# notice what does the send fucntion on two workers return; a multi-pointer
x_ptr


(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:82912183693 -> alice:42439475104]
	-> (Wrapper)>[PointerTensor | me:92240063561 -> bob:6010992038]

In [45]:
# what does .child on the pointer object return?
# what does .child.child on the pointer object return? 
x_ptr.child


MultiPointerTensor>{'alice': (Wrapper)>[PointerTensor | me:82912183693 -> alice:42439475104], 'bob': (Wrapper)>[PointerTensor | me:92240063561 -> bob:6010992038]}

In [46]:
x_ptr.child.child

{'alice': (Wrapper)>[PointerTensor | me:82912183693 -> alice:42439475104],
 'bob': (Wrapper)>[PointerTensor | me:92240063561 -> bob:6010992038]}

In [47]:
# try the .get() on the pointer
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [48]:
# try the .get(sum_results=True) on your pointer
x_ptr.get(sum_results=True)

AttributeError: ignored

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [50]:
x # a pointer to a remote tensor located at bob

(Wrapper)>[PointerTensor | me:41921784520 -> bob:73853469474]

In [51]:
y # a pointer to another remote tensor located at bob

(Wrapper)>[PointerTensor | me:72201357096 -> bob:27128834565]

In [52]:
z = x + y # treat those tensors (i.e., x and y) as local tensors, but they are actually executed remotely
z

(Wrapper)>[PointerTensor | me:59180784247 -> bob:88124798498]

In [53]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [54]:
z = th.add(x,y) # we can also use Torch operation on these tensors to be executed remotely
z

(Wrapper)>[PointerTensor | me:74784908724 -> bob:12594805276]

In [55]:
z = z.get() # yup! same results!
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [57]:
z = (x + y).sum()
z = z.get()
z

tensor(20., requires_grad=True)

In [0]:
z.backward() 

In [0]:
x = x.get()

In [60]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
x.grad

















# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). 

Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [63]:
import torch
# create some toy data for our model
input_data = th.tensor([[1.,1],[0.5,1],[1,0],[0,0]], requires_grad=True)
output_data = th.tensor([[1.],[1],[0],[0]], requires_grad=True)
input_data


tensor([[1.0000, 1.0000],
        [0.5000, 1.0000],
        [1.0000, 0.0000],
        [0.0000, 0.0000]], requires_grad=True)

In [0]:
# send the data to bob
x_ptr = input_data.send(bob)

In [65]:
# create some linear weights and send them to bob
weights = th.tensor([[0.],[1]], requires_grad=True)
weights

tensor([[0.],
        [1.]], requires_grad=True)

In [0]:
weights_ptr = weights.send(bob)

In [67]:
# create a linear model and train it on Bob's machine
# remember how to create a linear model? No? :( 
# Here's the main steps:
#    1- find a prediction
#    2- calcualte the loss (a mean square loss)
#    3- backpropogate using the backword() function
#    4- DO NOT forget to clear your gradients after updating the weights
#    weights.data.sum_(weights.grad * lr)
lr = 0.2
for i in range(10):
  prediction = torch.mm(input_data,weights)
  loss = (prediction - output_data)**2
  loss = loss.sum()
  loss.backward()
  print(loss)
  x = weights.data.sub_(weights.grad * lr)
  print(x)


tensor(0., grad_fn=<SumBackward0>)
tensor([[0.],
        [1.]])
tensor(0., grad_fn=<SumBackward0>)
tensor([[0.],
        [1.]])
tensor(0., grad_fn=<SumBackward0>)
tensor([[0.],
        [1.]])
tensor(0., grad_fn=<SumBackward0>)
tensor([[0.],
        [1.]])
tensor(0., grad_fn=<SumBackward0>)
tensor([[0.],
        [1.]])
tensor(0., grad_fn=<SumBackward0>)
tensor([[0.],
        [1.]])
tensor(0., grad_fn=<SumBackward0>)
tensor([[0.],
        [1.]])
tensor(0., grad_fn=<SumBackward0>)
tensor([[0.],
        [1.]])
tensor(0., grad_fn=<SumBackward0>)
tensor([[0.],
        [1.]])
tensor(0., grad_fn=<SumBackward0>)
tensor([[0.],
        [1.]])


# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects()

In [69]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [71]:
bob._objects

{11982058666: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [73]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [75]:
bob._objects

{13220092548: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [77]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [79]:
x

(Wrapper)>[PointerTensor | me:69749647791 -> bob:50778261790]

In [80]:
bob._objects

{50778261790: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [82]:
bob._objects  # some error from Jupyter is casuing this to appear!

{50778261790: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [84]:
bob._objects

{50778261790: tensor([1, 2, 3, 4, 5])}

In [85]:
bob = bob.clear_objects() # erase force 
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [87]:
bob._objects # notice that there is only a single tensor in bob

{90665792139: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [89]:
z = x + y

TensorsNotCollocatedException: ignored

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [91]:
z = x + y

TensorsNotCollocatedException: ignored

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [96]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(9.6414)
tensor(0.9236)
tensor(0.2075)
tensor(0.1129)
tensor(0.0775)
tensor(0.0553)
tensor(0.0398)
tensor(0.0288)
tensor(0.0210)
tensor(0.0154)
tensor(0.0114)
tensor(0.0085)
tensor(0.0063)
tensor(0.0047)
tensor(0.0035)
tensor(0.0027)
tensor(0.0020)
tensor(0.0015)
tensor(0.0011)
tensor(0.0009)


In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [101]:
train()

tensor(0.8450, requires_grad=True)
tensor(0.1070, requires_grad=True)
tensor(0.0663, requires_grad=True)
tensor(0.0802, requires_grad=True)
tensor(0.0302, requires_grad=True)
tensor(0.0490, requires_grad=True)
tensor(0.0174, requires_grad=True)
tensor(0.0297, requires_grad=True)
tensor(0.0103, requires_grad=True)
tensor(0.0181, requires_grad=True)
tensor(0.0062, requires_grad=True)
tensor(0.0112, requires_grad=True)
tensor(0.0037, requires_grad=True)
tensor(0.0069, requires_grad=True)
tensor(0.0023, requires_grad=True)
tensor(0.0044, requires_grad=True)
tensor(0.0014, requires_grad=True)
tensor(0.0028, requires_grad=True)
tensor(0.0009, requires_grad=True)
tensor(0.0018, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(0.0012, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0008, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0005, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0001, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [102]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [105]:
bob._objects

{71200034839: tensor([1, 2, 3, 4, 5])}

In [106]:
alice._objects

{2203755474: (Wrapper)>[PointerTensor | alice:2203755474 -> bob:71200034839]}

In [0]:
y = x + x

In [108]:
y

(Wrapper)>[PointerTensor | me:52325636857 -> alice:23680839155]

In [109]:
bob._objects

{60446487060: tensor([ 2,  4,  6,  8, 10]),
 71200034839: tensor([1, 2, 3, 4, 5])}

In [110]:
alice._objects

{2203755474: (Wrapper)>[PointerTensor | alice:2203755474 -> bob:71200034839],
 23680839155: (Wrapper)>[PointerTensor | alice:23680839155 -> bob:60446487060]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [113]:
bob._objects

{61089901530: tensor([1, 2, 3, 4, 5])}

In [114]:
alice._objects

{38918357759: (Wrapper)>[PointerTensor | alice:38918357759 -> bob:61089901530]}

In [115]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:38918357759 -> bob:61089901530]

In [116]:
bob._objects

{61089901530: tensor([1, 2, 3, 4, 5])}

In [117]:
alice._objects

{}

In [118]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [119]:
bob._objects

{}

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [121]:
bob._objects

{15264737796: tensor([1, 2, 3, 4, 5])}

In [122]:
alice._objects

{38675165444: (Wrapper)>[PointerTensor | alice:38675165444 -> bob:15264737796]}

In [0]:
del x

In [124]:
bob._objects

{}

In [125]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [126]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [128]:
bob._objects

{899032423: tensor([1, 2, 3, 4, 5])}

In [129]:
alice._objects

{}

In [130]:
x.move(alice)

(Wrapper)>[PointerTensor | me:8886171600 -> alice:8886171600]

In [131]:
bob._objects

{}

In [132]:
alice._objects

{8886171600: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [134]:
bob._objects

{64779076867: tensor([1, 2, 3, 4, 5])}

In [135]:
alice._objects

{8886171600: tensor([1, 2, 3, 4, 5]),
 50844131523: (Wrapper)>[PointerTensor | alice:50844131523 -> bob:64779076867]}

In [136]:
x.remote_get()

(Wrapper)>[PointerTensor | me:72707522788 -> alice:50844131523]

In [137]:
bob._objects

{}

In [138]:
alice._objects

{8886171600: tensor([1, 2, 3, 4, 5]), 50844131523: tensor([1, 2, 3, 4, 5])}

In [139]:
x.move(bob)

(Wrapper)>[PointerTensor | me:72707522788 -> bob:72707522788]

In [140]:
x

(Wrapper)>[PointerTensor | me:72707522788 -> bob:72707522788]

In [141]:
bob._objects

{72707522788: tensor([1, 2, 3, 4, 5])}

In [142]:
alice._objects

{8886171600: tensor([1, 2, 3, 4, 5])}